In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.datasets import load_files

In [3]:
# 데이터 폴더 설정 후 읽어온다.
reviews = load_files('/content/drive/MyDrive/인공지능사관학교/빅데이터/03_확률 모델링_업데이트/data/txt_sentoken')
my_docs, y = reviews.data, reviews.target

In [4]:
my_docs_0  = list(pd.Series(my_docs)[y==0])

my_docs_1  = list(pd.Series(my_docs)[y==1])
print(f"0 유형의 수 : {len(my_docs_0)}")
print(f"1 유형의 수 : {len(my_docs_1)}")

0 유형의 수 : 1000
1 유형의 수 : 1000


In [43]:
n_train = 600
n_test  = 1000 - n_train

my_docs_train_0 = my_docs_0[:n_train]
my_docs_train_1 = my_docs_1[:n_train]
my_docs_test_0 = my_docs_0[n_train:]
my_docs_test_1 = my_docs_1[n_train:]
my_docs_test = my_docs_0[n_train:] + my_docs_1[n_train:]
Y_test = [0]*n_test + [1]*n_test

In [45]:
print(my_docs_train_0)
print(Y_test)

Output hidden; open in https://colab.research.google.com to view.

In [44]:
# 전처리 해서 단어별 도수표를 만들어주는 함수.
def preprocessor(a_doc):
    freq_dict = {}
    for a_line in a_doc:
        a_line = str(a_line)                  # 단순한 문자열로 변환.
        a_line = a_line.lower()               # 소문자화.
        a_line = re.sub(r"\W"," ",a_line)     # 특수문자 제거.
        a_line = re.sub(r"\d", " ", a_line)   # 숫자 제거.
        a_line = re.sub("a|the|and|or|because|at|the|we|he|she", " ",a_line)  #  불용어 제거.
        a_line = a_line.split()               # 분절.
        for a_word in a_line:
            if len(a_word) > 3:               # 길이가 최소 조건을 충족하는 단어만 사용.            
                if a_word in freq_dict:
                    freq_dict[a_word] += 1    # 카운트 누적.
                else:
                    freq_dict[a_word] = 2     # 처음 발견. 기본값 1 + 누적 1.
    return freq_dict

In [46]:
# Series로 변환.
freq_0 = pd.Series(preprocessor(my_docs_train_0)).sort_values(ascending=False)
freq_1 = pd.Series(preprocessor(my_docs_train_1)).sort_values(ascending=False) 

In [47]:
# Vocabulary 사전 크기 확인.
print("Size of 0: {}".format( len(freq_0)))
print("Size of 1: {}".format( len(freq_1)))

Size of 0: 15976
Size of 1: 16811


In [48]:
# Vocabulary 수 맞춤.
n_voca = 19000
freq_0 = freq_0.iloc[:n_voca]
freq_1 = freq_1.iloc[:n_voca]

In [53]:
print(freq_0)
print(freq_1)

with         2926
film         2574
this         2455
movie        1952
from         1326
             ... 
nlines          2
cryb            2
gines           2
nsiouxsie       2
ditions         2
Length: 15976, dtype: int64
with       3480
film       3170
this       2368
from       1632
movie      1555
           ... 
seer          2
poets         2
nquiet        2
ipkiss        2
donohue       2
Length: 16811, dtype: int64


In [49]:
# 로그 확률로 변환.
freq_0_sum = freq_0.sum()
freq_1_sum = freq_1.sum()
log_prob_0 = dict(np.log(freq_0/freq_0_sum))    # Log(P(W|0))
log_prob_1 = dict(np.log(freq_1/freq_1_sum))    # Log(P(W|1))

In [54]:
print(freq_0_sum)
print(freq_1_sum)

182277
204394


In [52]:
print(log_prob_0)
print(log_prob_1)

{'with': -4.13189120546613, 'film': -4.260066398890127, 'this': -4.307400746817579, 'movie': -4.536673020073162, 'from': -4.923360616300392, 'like': -5.079776302758503, 'bout': -5.123947521071639, 'just': -5.309777529437862, 'tion': -5.3243110440540296, 'which': -5.334497889361023, 'some': -5.334497889361023, 'only': -5.434940672391868, 'into': -5.474714997879679, 'even': -5.521609055037541, 'good': -5.565063684283827, 'time': -5.589720480896688, 'been': -5.6332382251195465, 'would': -5.63477314483763, 'much': -5.662812364902023, 'cter': -5.750254683505735, 'will': -5.785346003317005, 'plot': -5.794314673299765, 'nbut': -5.851791102725158, 'fter': -5.87300694187543, 'cters': -5.8887243577708395, 'most': -5.910747269858277, 'could': -5.9332661333936265, 'ction': -5.954187398554267, 'first': -5.966953529377302, 'nthis': -5.969097152920554, 'king': -5.979884744049551, 'over': -5.986413602932014, 'little': -5.997390661563165, 'does': -6.019713017001064, 'ting': -6.026508060133892, 'very': 

In [50]:
Y_pred = []
for a_sentence in my_docs_test:
    log_prob_sum_0 = 0.
    log_prob_sum_1 = 0.
    a_sent_preprocessed = preprocessor([a_sentence])
    for a_word, a_freq in a_sent_preprocessed.items():
        if a_word in log_prob_0:
            log_prob_sum_0 += log_prob_0[a_word]
        else:
            log_prob_sum_0 += np.log(1.0/freq_0_sum)
            
        if a_word in log_prob_1:
            log_prob_sum_1 += log_prob_1[a_word]
        else:
            log_prob_sum_1 += np.log(1.0/freq_1_sum)
            
    if (log_prob_sum_0 > log_prob_sum_1):
        Y_pred.append(0)
    else:
        Y_pred.append(1)
    

'''
빈도수가 아니라 단어가 출연 했는 것은 이미 확률 계산을 통해 반영을 했기 때문에 
a_freq 빈도수를 반영할 필요는 없다.
'''

In [51]:
# 정확도 계산.
correct = pd.Series([ x == y for (x,y) in zip(Y_pred, Y_test) ])
print("Accuracy : {:.3f}".format(correct.mean()))

Accuracy : 0.805
